##Some special types of testing

###Why don't numerical test pass?

Python has inherent error when doing floating point calculations:

In [1]:
0.1 + 0.2

0.30000000000000004

Leading to situations like this:

In [2]:
assert(0.1 + 0.2 == 0.3)

AssertionError: 

For many functions, this isn't a problem in real life but makes it hard to test. Solution: `test_almost_equal()`:


In [3]:
from nose.tools import assert_almost_equal
assert_almost_equal(0.1 + 0.2, 0.3)

###How do we test exceptions?

Imagine that we want our common k-gram function to raise a `ValueError` if we give a value for n of less than 1:

In [6]:
def find_common_ngrams(text, cutoff, n):
    if n<1:
        raise ValueError('n must be at least 1')
    words = text.lower().split(' ')
    ngram_counts = {}
    for start in range(len(words) +1 - n):
        ngram = ' '.join(words[start:start+n])
        current_count = ngram_counts.get(ngram, 0)
        ngram_counts[ngram] = current_count + 1
    
    result = []
    for ngram, count in ngram_counts.items():
        if count >= cutoff:
            result.append(ngram)
        
    return result

text = "it was the best of times it was the worst of times"
find_common_ngrams(text, 8, 0)

ValueError: n must be at least 1

How to test it? `assert` and `assert_equal` won't work, because the function doesn't actually return a value. Solution: use `assert_raises`:

In [7]:
from nose.tools import assert_raises
assert_raises(ValueError, find_common_ngrams, text, 8, 0)

###How to deal with tests that alter state?

The `find_common_ngrams()` function is easy to test because it doesn't change the *state* of the program. 

Consider this function, which takes a list of words and removes any long ones:

In [8]:
def remove_long_words(word_list, max):
    for word in list(word_list):
        if len(word) > max:
            word_list.remove(word)
            
words = "it was the best of times it was the worst of times".split()
print(words)
remove_long_words(words, 2)
print(words)

['it', 'was', 'the', 'best', 'of', 'times', 'it', 'was', 'the', 'worst', 'of', 'times']
['it', 'of', 'it', 'of']


How to test it? This well be fine at first:

In [11]:
from nose.tools import assert_equal

def remove_long_words(word_list, max):
    for word in list(word_list):
        if len(word) > max:
            word_list.remove(word)
            
words = "it was the best of times it was the worst of times".split()

def test_two():
    remove_long_words(words, 2)
    assert_equal(words, ['it', 'of', 'it', 'of'])

But then when we add another test: danger!

In [12]:
from nose.tools import assert_equal

def remove_long_words(word_list, max):
    for word in list(word_list):
        if len(word) > max:
            word_list.remove(word)
            
words = "it was the best of times it was the worst of times".split()

def test_two():
    remove_long_words(words, 2)
    assert_equal(words, ['it', 'of', 'it', 'of'])

def test_four():
    remove_long_words(words, 4)
    assert_equal(words, ['it', 'was', 'the', 'best', 'of', 'it', 'was', 'the', 'of'])

Now the sucess/failure of the tests depends on their order. If `test_two()` is run before `test_four()` then `test_four()` will fail because all the 3 and 4 character words will already be removed. 

**Having tests depend on their order is a Bad Idea!**

We need to reset the `words` variable back to the starting point each time. We can do it inside each test function:

In [13]:
def test_two():
    words = "it was the best of times it was the worst of times".split()
    remove_long_words(words, 2)
    assert_equal(words, ['it', 'of', 'it', 'of'])

def test_four():
    words = "it was the best of times it was the worst of times".split()
    remove_long_words(words, 4)
    assert_equal(words, ['it', 'was', 'the', 'best', 'of', 'it', 'was', 'the', 'of'])

This only works as long as the set-up code is short. `nose` has a more general solution:

In [14]:
from nose.tools import assert_equal, with_setup

def remove_long_words(word_list, max):
    for word in list(word_list):
        if len(word) > max:
            word_list.remove(word)         

words = []
def setup_words():
    global words
    words = "it was the best of times it was the worst of times".split()

@with_setup(setup_words)
def test_two():    
    remove_long_words(words, 2)
    assert_equal(words, ['it', 'of', 'it', 'of'])

@with_setup(setup_words)
def test_four():
    remove_long_words(words, 4)
    assert_equal(words, ['it', 'was', 'the', 'best', 'of', 'it', 'was', 'the', 'of'])

Create a set-up function, and pass the name of the function to a decorator for tests that require it. Why is this better?

- set-up code and test code are separated, making them both easier to read
- set-up code doesn't have to be repeated
- we can see at a glance which tests require set-up

In [222]:
# ignore this cell, it's for loading custom js code
from IPython.core.display import Javascript
Javascript(filename="custom.js")

<IPython.core.display.Javascript object>

In [223]:
# ignore this cell, it's for loading custom css code
from IPython.core.display import HTML
HTML(filename="custom.css")